In [4]:
# two convolutional layers model

model_conv = nn.Sequential(
    nn.Conv2d(10, 10, 3),
    nn.BatchNorm2d(10),
    nn.Conv2d(10, 10, 3),
    nn.BatchNorm2d(10)
)


inputs = torch.randn((120, 10, 512, 512))
inputs = inputs.to('cuda')
model_conv.to('cuda')

start_time = time.time()
outputs = model_conv(inputs)
end_time = time.time()
print('model_conv using time {:.4f} s'.format(end_time-start_time))
del inputs
del outputs
del model_conv
torch.cuda.empty_cache()

model_conv using time 0.0141 s


In [ ]:
from __future__ import print_function
import numpy as np
import time
import torch
import torch.nn as nn
from collections import OrderedDict

def get_mask(in_channels, channels):
    mask = np.zeros((in_channels, channels, 3, 3))
    for _ in range(in_channels):
        mask[_, _ % channels, :, :] = 1.
    return mask


class DiagonalwiseRefactorization(nn.Module):
    def __init__(self, in_channels, stride=1, groups=1):
        super(DiagonalwiseRefactorization, self).__init__()
        channels = int(in_channels / groups)
        self.in_channels = in_channels
        self.groups = int(groups)
        self.stride = stride
        self.mask = nn.Parameter(torch.Tensor(get_mask(in_channels, channels)), requires_grad=False)
        self.weight = nn.Parameter(torch.Tensor(in_channels, channels, 3, 3), requires_grad=True)
        torch.nn.init.xavier_uniform_(self.weight.data)
        self.weight.data.mul_(self.mask.data)

    def forward(self, x):
        weight = torch.mul(self.weight, self.mask)
        x = torch.nn.functional.conv2d(x, weight, bias=None, stride=self.stride, padding=1, groups=self.groups)
        return x


def DepthwiseConv2d(in_channels, stride=1):
    groups = max(in_channels / 10, 1)
    return DiagonalwiseRefactorization(in_channels, stride, groups)

In [6]:
# two depth-wise convolutional layers model 

model_dw_conv = nn.Sequential(
    nn.Conv2d(10, 10, 3, groups=10),
    nn.BatchNorm2d(10),
    nn.Conv2d(10, 10, 3, groups=10),
    nn.BatchNorm2d(10)
)

inputs = torch.randn((120, 10, 512, 512))
inputs = inputs.to('cuda')
model_dw_conv.to('cuda')

start_time = time.time()
outputs = model_dw_conv(inputs)
end_time = time.time()
print('model_conv using time {:.4f} s'.format(end_time-start_time))
# del inputs
# del outputs
del model_dw_conv
torch.cuda.empty_cache()

model_conv using time 0.0062 s


In [34]:
# two depth-wise convolutional layers model with Diagonalwise Refactorization
# https://arxiv.org/pdf/1803.09926.pdf
# https://github.com/clavichord93/diagonalwise-refactorization-pytorch
model_dw_conv = nn.Sequential(
    DepthwiseConv2d(10, stride=1),
    nn.BatchNorm2d(10),
    DepthwiseConv2d(10, stride=1),
    nn.BatchNorm2d(10)
)

inputs = torch.randn((120, 10, 512, 512))
inputs = inputs.to('cuda')
model_dw_conv.to('cuda')

start_time = time.time()
outputs = model_dw_conv(inputs)
end_time = time.time()
print('model_conv using time {:.4f} s'.format(end_time-start_time))
del inputs
del outputs
torch.cuda.empty_cache()

model_conv using time 0.0078 s


In [5]:
del inputs
del outputs
torch.cuda.empty_cache()